In [5]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor
from transformers import DeiTForImageClassification

from PIL import Image
import os
import torch.optim as optim
from tqdm import tqdm

2024-08-15 22:38:41.524690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-15 22:38:41.547554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-15 22:38:41.547624: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-15 22:38:41.563533: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-15 22:38:42.412991: W tensorflow/compiler/tf

In [6]:
image_folders = [
    "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Facebook",
    "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Instagram",
    "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Instagram",
    "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Telegram",
]

In [7]:


# Define a custom dataset class
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Set paths and labels


image_paths = []
labels = []

# Assume folder1 has label 0, folder2 has label 1, etc.
for i, folder in enumerate(image_folders):
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        image_paths.append(img_path)
        labels.append(i)

# Define transformations and dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = ImageDataset(image_paths, labels, transform=transform)
train_loader = DataLoader(dataset, batch_size=512, shuffle=True)


In [3]:
# from transformers import DeiTForImageClassification
from transformers import DeiTForImageClassification
model_name_or_path = 'google/vit-base-patch16-224-in21k'

# Clear cache and retry
model=DeiTForImageClassification.from_pretrained(model_name_or_path,num_labels=4,ignore_mismatched_sizes=True,force_download=True)
model

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

You are using a model of type vit to instantiate a model of type deit. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of DeiTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.distillation_token']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DeiTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized because the shapes did not match:
- embeddings.position_embeddings: found shape torch.Size([1, 197, 768]) in the checkpoint and torch.Size([1, 198, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DeiTForImageClassification(
  (deit): DeiTModel(
    (embeddings): DeiTEmbeddings(
      (patch_embeddings): DeiTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DeiTEncoder(
      (layer): ModuleList(
        (0-11): 12 x DeiTLayer(
          (attention): DeiTSdpaAttention(
            (attention): DeiTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): DeiTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): DeiTIntermediate(
            (dense): L

In [8]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model.classifier = torch.nn.Linear(model.config.hidden_size, len(image_folders))  # Adjusting output layer to match number of classes
model.to(device)


# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

100%|██████████| 6/6 [02:55<00:00, 29.28s/it]


Epoch [1/10], Loss: 1.3818


100%|██████████| 6/6 [02:43<00:00, 27.33s/it]


Epoch [2/10], Loss: 1.3512


100%|██████████| 6/6 [02:52<00:00, 28.69s/it]


Epoch [3/10], Loss: 1.2981


 67%|██████▋   | 4/6 [01:56<00:58, 29.05s/it]

In [ ]:
# Set test path
test_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Test"

# Define the test image folders and corresponding labels
image_folders = [
    "Facebook",
    "Instagram",
    "Telegram",
    "Whatsapp",
]
test_image_paths = []
test_labels = []

# Loop through each folder and collect image paths and labels
for i, folder in enumerate(image_folders):
    folder_path = os.path.join(test_path, folder)
    if os.path.exists(folder_path):
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            test_image_paths.append(img_path)
            test_labels.append(i)
    else:
        print(f"Warning: {folder_path} does not exist.")

# Define transformations for the test set
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create test dataset and dataloader
test_dataset = ImageDataset(test_image_paths, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)


In [ ]:
# Save the trained model
# torch.save(model.state_dict(), "facebook_vit_classification_model.pth")

# Evaluate the model (optional)
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

# If you have a separate validation set, use the evaluate_model function
# For now, evaluating on the same training set
evaluate_model(model, test_loader)


In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model.classifier = torch.nn.Linear(model.config.hidden_size, len(image_folders))  # Adjusting output layer to match number of classes
model.to(device)


# Define the optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 10
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

100%|██████████| 6/6 [02:55<00:00, 29.28s/it]


In [ ]:
# import torch
# from torch.utils.data import DataLoader, Dataset
# from torchvision import transforms
# from transformers import DeiTForImageClassification, DeiTFeatureExtractor
# from PIL import Image
# import os
# import torch.optim as optim
# from tqdm import tqdm

# # Define a custom dataset class
# class ImageDataset(Dataset):
#     def __init__(self, image_paths, labels, transform=None):
#         self.image_paths = image_paths
#         self.labels = labels
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         image = Image.open(self.image_paths[idx]).convert("RGB")
#         label = self.labels[idx]
#         if self.transform:
#             image = self.transform(image)
#         return image, label

# # Set paths and labels
# image_folders = [
#     "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Facebook",
#     "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Instagram",
#     "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Instagram",
#     "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/cv-learnings/coco_ankiita/SD2Q/Train/Telegram",
# ]

# image_paths = []
# labels = []

# # Assume folder1 has label 0, folder2 has label 1, etc.
# for i, folder in enumerate(image_folders):
#     for img_name in os.listdir(folder):
#         img_path = os.path.join(folder, img_name)
#         image_paths.append(img_path)
#         labels.append(i)

# Define transformations and dataset
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# dataset = ImageDataset(image_paths, labels, transform=transform)
# data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# # Load the DeiT-Tiny model and move it to the GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# from transformers import DeiTForImageClassification

# Clear cache and retry
DeiTForImageClassification.from_pretrained('facebook/deit-tiny-patch16-224', force_download=True)

model.classifier = torch.nn.Linear(model.config.hidden_size, len(image_folders))  # Adjusting output layer to match number of classes
model.to(device)

# # Define the optimizer and loss function
# optimizer = optim.AdamW(model.parameters(), lr=5e-4)
# criterion = torch.nn.CrossEntropyLoss()

# # Training loop
# num_epochs = 10
# model.train()

# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for images, labels in tqdm(data_loader):
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(images)
#         loss = criterion(outputs.logits, labels)

#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     # Print epoch loss
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader):.4f}")

# # Save the trained model
# torch.save(model.state_dict(), "deit_tiny_classification_model.pth")

# # Evaluate the model (optional)
# def evaluate_model(model, data_loader):
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for images, labels in data_loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             _, predicted = torch.max(outputs.logits, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     print(f'Accuracy: {100 * correct / total:.2f}%')

# # If you have a separate validation set, use the evaluate_model function
# # For now, evaluating on the same training set
# evaluate_model(model, data_loader)


In [ ]:
# Define your custom model
model = create_crossvit_model(input_shape=(224, 224, 7), num_classes=4)